# OGGM Glacier Panel

HoloViews+Panel implementation of a [dashboard originally developed in Plotly+Dash](https://github.com/OGGM/OGGM-Dash/blob/master/apps/explore/app.py).  The dashboard can be used here in the notebook, or you can run it as a separate server using:

    panel serve OGGM_Glacier.ipynb --show

In [ ]:
import os
import numpy as np
import dask.dataframe as dd
import cartopy.crs as ccrs

import bokeh
import panel as pn
import holoviews as hv
import geoviews as gv

hv.extension('bokeh')

### Preprocess data

In [ ]:
df = dd.read_csv('data/oggm_glacier_explorer.csv').persist()
df['latdeg'] = df.cenlat

ds = gv.Points(df, [('cenlon', 'Longitude'), ('cenlat', 'Latitude')],   # key dimensions
                    [('avg_prcp', 'Annual Precipitation (mm/yr)'), # value dimensions
                     ('area_km2', 'Area'), ('latdeg', 'Latitude (deg)'),
                     ('avg_temp_at_mean_elev', 'Annual Temperature at avg. altitude'), 
                     ('mean_elev', 'Elevation')])
ds = gv.operation.project_points(ds).map(gv.Dataset, gv.Points).clone(crs=ccrs.GOOGLE_MERCATOR)

precip_range = ds.range('avg_prcp')
temp_range = ds.range('avg_temp_at_mean_elev')

### Static header items

Defines static items to use in the header at the top of the dashboard.  If you want more control over the formatting, could define these items in a separate Jinja2 template.

In [ ]:
title = '<p style="font-size:35px">World glaciers explorer</p>'
instruction = 'Box-select on each plot; make an empty selection on the same plot to reset.'
oggm_logo = '<a href="https://github.com/OGGM/OGGM-Dash"><img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logos/oggm_s_alpha.png" width=170></a>'
pv_logo = '<a href="https://pyviz.org"><img src="http://pyviz.org/assets/PyViz_logo_wm.png" width=80></a>'
clear_button = pn.widgets.Button(name='Clear selection', width=100)

### Declare and link plots

The dashboard contains several plots, each linked so that selections in one plot constrain the data selected in the others. Notes on the implementation:

* Because all plots derive from a single underlying data source, we will first declare an `hv.Dataset` object where we can label the key (independent) and value (dependent) dimensions in a way that will be inherited by each individual plot. 
* Static versions of each plot allow data to be selected and form the background on which those selections will be displayed
* Selections are implemented as HoloViews/Bokeh streams, with ``Selection1D`` streams for the points (selecting in a box) and ``BoundsX`` streams for the histograms (selecting on one axis).
* The various selections across each plot are combined using the ``combine_selections`` function
* The selections form a dynamic version of each static plot, overlaid on the static plot.
* An additional widget makes it simple to clear all selections without having to select each plot in turn.

In [ ]:
from holoviews.streams import Stream, BoundsXY, BoundsX
from bokeh.palettes import plasma
from bokeh.models import HoverTool

def combine_selections(kwargs):
    """
    Combines selections on all 4 plots into a single selection by index.
    """
    if all(not v for v in kwargs.values()):
        return slice(None)
    selection = {}
    for key, bounds in kwargs.items():
        if bounds is None:
            continue
        elif len(bounds) == 2:
            selection[key[:-7]] = bounds
        else:
            xbound, ybound = key.split('__')
            selection[xbound] = bounds[0], bounds[2]
            selection[ybound[:-7]] = bounds[1], bounds[3]
    return sorted(set(ds.select(**selection).data.index))

color_precip = '#85c1e9'
color_temp = '#f1948a'
color_lat = '#7d3c98'

def select_ds(**kwargs):
    return ds.iloc[combine_selections(kwargs)] if kwargs else ds

def count(ds):
    return hv.Div('<p style="font-size:20px">Glaciers selected: ' + 
                  str(len(ds)) + "</font>").options(height=40)

def geo_view(ds):
    return gv.Points(ds, crs=ccrs.GOOGLE_MERCATOR).options(alpha=1)

def lat_alt_view(ds):
    return ds.to(hv.Scatter, 'mean_elev', 'latdeg', [])

hist_kwargs  = dict(num_bins=50, adjoin=False, normed=False)
hist_options = dict(width=600, default_tools=[], toolbar=None, alpha=1.0)

def temp_hist(ds):
    return ds.hist('avg_temp_at_mean_elev', bin_range=temp_range, **hist_kwargs).options(
        fill_color=color_temp, **hist_options)

def precip_hist(ds):
    return ds.hist('avg_prcp', bin_range=precip_range, **hist_kwargs).options(
        fill_color=color_precip, **hist_options)

import datashader as dsr
from holoviews.util import Dynamic
from holoviews.operation.datashader import rasterize, datashade

# Static views
geopoints = ds.to(gv.Points, ['cenlon', 'cenlat'], ['area_km2'], []).redim.range(area_km2=(0, 3000))
geo_options = dict(width=600, height=500, global_extent=True, tools=['hover', 'box_select'], alpha=0.1, cmap='viridis', logz=True)
shaded_points = rasterize(geo_view(ds), aggregator=dsr.sum('area_km2'), x_sampling=5000, y_sampling=5000).options(**geo_options)
static_lat_alt     = datashade(lat_alt_view(ds), cmap=plasma(256)).options(alpha=0.1, tools=['box_select'], width=600, height=500, show_grid=True)
static_temp_hist   = temp_hist(ds).options(alpha=0.1)
static_precip_hist = precip_hist(ds).options(alpha=0.1)

# Selections
geo_selection = BoundsXY(source=shaded_points,      rename={'bounds': 'cenlon__cenlat_bounds'})
alt_selection = BoundsXY(source=static_lat_alt, rename={'bounds': 'mean_elev__latdeg_bounds'})
temp_bounds   = BoundsX(source=static_temp_hist,   rename={'boundsx': 'avg_temp_at_mean_elev_bounds'})
precip_bounds = BoundsX(source=static_precip_hist, rename={'boundsx': 'avg_prcp_bounds'})
selections    = [geo_selection, alt_selection, temp_bounds, precip_bounds]

# Dynamically selected views

ds_dmap = hv.DynamicMap(select_ds, streams=selections)
geopoint_dmap    = rasterize(Dynamic(ds_dmap, operation=geo_view), aggregator=dsr.sum('area_km2'),
                             x_sampling=5000, y_sampling=5000).options(cmap='viridis', logz=True)
lat_alt_dmap     = datashade(Dynamic(ds_dmap, operation=lat_alt_view), cmap=plasma(256))
temp_hist_dmap   = Dynamic(ds_dmap, operation=temp_hist)
precip_hist_dmap = Dynamic(ds_dmap, operation=precip_hist)
count_dmap       = Dynamic(ds_dmap, operation=count)

# Combined views
map_          = shaded_points * geopoint_dmap * gv.tile_sources.Wikipedia.options(alpha=0.4)
altitude      = static_lat_alt*lat_alt_dmap
temperature   = static_temp_hist*temp_hist_dmap
precipitation = static_precip_hist*precip_hist_dmap

# Button to reset selections
clear_button = pn.widgets.Button(name='Clear selection')
def clear_selections(event):
    geo_selection.update(bounds=None)
    alt_selection.update(bounds=None)
    temp_bounds.update(boundsx=None)
    precip_bounds.update(boundsx=None)
    Stream.trigger(selections)

clear_button.param.watch(clear_selections, 'clicks');

## Construct dashboard

Combine a header and the plots into a dashboard displayed in the notebook and serveable on bokeh server:

In [ ]:
header = pn.Row(pn.Pane(oggm_logo, width=170), pn.Spacer(width=50), pn.Column(pn.Pane(title, height=25, width=400), pn.Spacer(height=-15), pn.Pane(instruction, width=500)),
                pn.Spacer(width=180), pn.Column(pn.Pane(count_dmap), clear_button, pn.Spacer(height=-15)), 
                pn.Pane(pv_logo, width=80))

pn.Column(header, pn.Spacer(height=-50), pn.Row(map_, altitude), pn.Row(temperature, precipitation)).servable()